## 1. Attention And Transformer

### Seq2seq
입력으로 일련의 단어들이 들어오고 이를 이용해서 다시 일련의 단어들을 생성해야 하는 문제를 말한다.
**과정**
문장에 대한 내용을 이해 → 정해진 형태(벡터)로 이해된 내용 저장 → 번역의 첫 단어 예측(사전에 있는 단어들에 대한 확률 계산) → 높은 확률의 단어 제안
→ 벡터와 첫 단어를 통해 다음 단어 예측 → 반복

**구조**
Encoder and Decoder
예를 들어 번역으로 보자면, (영어 → 한국어)
Encoder : 영어 문장을 이해하는 역할 (번역하고자 하는 문장)

은닉층을 통해 앞 단어로부터 순차적으로 정보가 축적되고 마지막 <end> 입력을 받은 은닉층의 노드는 영어 문장 전체의 문맥 정보를 내포한다.
→ **마지막 노드**에 전체적인 정보가 들어가 있다.

Decoder : 이 문맥정보로부터 한국어 문장을 생성하는 역할을 한다.
<start> : 문장의 시작 혹은 번역의 시작을 알리는 벡터
디코더는 한 번에 실행되지 않고 각 단어를 예측하는 단계가 **순차적으로** 실행된다.
<end> : 번역을 마친다.

결론 : Encoder가 문맥정보를 축적하면, Decoder는 이 문맥정보에 자신이 생성한 단어들을 하나씩 결합하면서 다음 단어를 예측하게 되고,
생성해야 할 단어의 수만큼 모형을 이용한 예측이 반복된다.

### Attention을 이용한 성능 향상

#### seq2seq의 문제점
문맥 정보가 인코더의 마지막 벡터 하나에 집중되는 현상이 나타난다.
이 하나의 벡터에 번역할 무장의 모든 문맥 정보가 포함되고, 이를 이용해 디코더에서 반복적으로 다음 단어를 예측한다.
만약, 번역해야할 문장이 엄청 길다면, 하나의 벡터에 많은 정보가 잘 축적되고 또 이를 디코더에서 풀어내기가 어렵다. (성능이 떨어짐)

#### Attention
하나의 단어가 단어를 번역할 때 직접 관여할 수 있도록 한다.
Encoder의 윗부분에 **context vector**를 생성한다.
이 벡터는 첫 단어의 예측에 가장 많은 영향을 미치는 단어에 대한 정보가 담겨있다.
단어별로 **Attention Score**를 계산하고 softmax를 적용한 가중치를 표현한다. (가중치가 가장 높은 단어가 현재 예측해야할 단어에 가장 영향을 많이 미치는 것)
이 단어는 학습을 통해 결정된다.

결론 : 단어들의 임베딩 벡터에 대해 가중치를 반영해 가중 합계를 구하면, context vector가 되고 이 context vector가 encoder의 마지막 벡터와 deoder의 입력값과 합쳐져 첫 단어를 생성한다. **context vecotr는 당연히 예측할 단어의 순서에 따라 계속 변화한다.**

### Self-Attention

예시. The animal didn't cross the street because it was too tired.
VS The animal didn't cross the street because it was too wide.

여기서 it이 가리키는 명사는 다르다. (animal - street)
이와 같이 동일한 단어라도 **문맥에 따라 다른 의미를 갖게 되고** 그 의미에 영향을 미치는 단어가 문장에 존재한다.

셀프 어텐션의 목적 : 문장 내에서 단어 간 영향을 표현하는 것
어떤 단어를 벡터로 임베딩할 때, 그 단어에 영향을 미치는 다른 단어들의 정보를 **함께 인코딩**하는 것이다.
(각 단어들에 대해 그 단어에 영향을 미치는 단어들의 정보를 선별해(가중치) 자신에게 축적한다.)
→ 이렇게 되면, 어느 한 벡터가 전체 문맥에 대한 정보를 축적하고 있지 않다. (모두 자기 자신에 대한 벡터를 가지고 있음)
Encoding 과정에서 문맥에 대한 정보는 각 단어에 골고루 분포하고 Decoding 과정에서는 입력 문장의 **모든 단어의 임베딩 벡터**를 어텐션 형태로 사용한다.


### Transformer

Transformer는 기존 seq2seq 모형에서 RNN 혹은 LSTM 구조를 다 버리고 오직 Attention에만 의지한 모형을 제안하였다.

#### Transformer 구조
Encoder : 번역의 대상이 되는 문장을 입력받아 문맥 정보, 즉 셀프 어텐션 정보를 추출한다.
Decoder : 셀프 어텐션 정보를 갖고 있는 각 단어의 임베딩 벡터를 이용해 단어를 하나씩 예측한다.
이 때, Encoder로부터 오는 정보뿐만 아니라 자신의 셀프 어텐션 정보를 함께 사용한다.

1 . Transformer 기반의 토큰화

- 문자 기반 토큰화
- BPE
- WordPiece
- SentencePiece

출력 sequence에서도 동일하게 토큰 임베딩이 수행된다.
예를 들어 "Attention is all you need"인 경우,
첫번째 예측 : Attention → "토큰 임베딩 : Attention"
두번째 예측 : is → "토큰 임베딩 : Attention is"
세번째 예측 : all → "토큰 임베딩 : Attention is all" 이런식으로 이루어지고 그 다음 단어를 예측한다.

2 . 위치 인코딩 (Positional Encoding)

트랜스포머는 모든 토큰 사이에 동등한 self-attention이 동작하기 때문에 토큰의 순서와 관련이 없는 모형이다.
따라서, 위치에 대한 정보를 토큰에 추가하는 작업을 수행한다.

3 . n회 반복

층이 겹쳐져 있는 경우, 앞 층의 결과는 다음 층의 입력으로 사용된다.
하나의 층은 멀티헤드 어텐션과 피드포워드 신경망으로 구성되어 있다.

**multi-head-attention** : self-attention이 병렬로 여러 개 사용되는 것
(다양한 관점에서 self-attention을 구현하고 이를 결합해 최종 attention을 만든다.)
이 과정을 통해 단어에 영향을 미치는 단어들의 정보가 결합된 벡터가 추출된다. (단어의 정보 크기 조절)
→ 예를 들어, 서로 다른 여러 사람이 각자의 관점으로 문장을 해석하고, 그 의견들을 취합하는 과정

feed-forward-neural-network : 단어들의 정보 벡터를 통해 출력한다. (이는 다음 인코더 층에 입력으로 사용된다.)

이 2개의 신경망 사이의 **잔차연결&정규화 층**: 이 층에서 잔차연결과 레이어 정규화가 이루어진다.
잔차연결 : self-attention 과정을 거치면서 원래 embedding vector의 정보가 지나치게 손실 혹은 변형되는 것을 막는 역할
multi-attention을 통과한 출력값과 본래 embedding vector가 결합하면서 본래 embeddig vector의 정보가 어느정도 유지될 수 있도록 한다.
정규화 : layer의 출력 데이터에 대해 평균과 분산을 이용하여 정규화를 수행함으로써 데이터를 안정화하고 학습속도를 개선한다.

#### Encoder의 self-attention 원리

셀프 어텐션은 query, key, value인 3개의 벡터로 이루어져 있다.
이 벡터들은 입력-단어의 임베딩 벡터로부터 각각의 가중치 행렬로 계산되어 나온다.

query :  어텐션을 받는 단어(현재 주목을 받는 단어)가 어텐션과 고나련해 다른 단어들에게 던지는 질문 .. ?
예시. 나에게 영향을 미치는 단어들 말해주세요 !
key : 질문에 대한 대답
예시. 제가 이정도로 당신에게 영향을 미칩니다.
value : 어텐션을 받는 단어가 다른 단어에게 영향을 받는 정도 (내적을 통해 자신에게 오는 영향의 정도를 계산)

정보량을 다 더함으로써 어텐션을 받는 단어에 대한 Attention vector가 오나성된다.
나에게 가장 영향을 많이 주는 단어는 나 자신이지만, 자신 외에 나에게 영향을 미치는 단어들의 정보가 포함되면서 문장의 의미적인 구조인 문맥을 파악할 수 있다.

#### Decoder의 동작 원리

Decoder는 출력 단어 시퀀스의 첫 토큰 <start>로 시작하여 단어가 순차적으로 추가되는 시퀀스를 입력으로 받는다.

Decoder는 Encoder와 달리 **mask multi-head attention**, 잔차연결&정규화의 2단계를 수행한 이후,
인코더로부터 연결되는 Encoder-Decoder Attention, 잔차연결&정규화의 2단계가 추가된다.

1 . **mask multi-head attention**

Decoder의 특성상 self-attention이 뒤에서 앞으로 갈 수 없기 때문에 만들어진 매커니즘이다. (하나의 단어 예측 → 다음 단어 예측)
첫번째 단어를 예측한 뒤, 다음 단어를 예측하기 때문에 첫째 단어에서 둘째 단어로만 영향을 미친다. (미래에서 영향을 받는 것은 당연히 어렵다.)
이와 같이 Decoder에서 순방향으로만 Attention이 향하는 것을 구현한 매커니즘이다.

2 . Encoder and Decoder Multihead Attention

Encoder에서 Decoder로 향하는 어텐션을 구현한 층이다.
query를 던지는 단어는 디코더에서 생성하고 있는 단어이다. (영어에서 한국어로 번역을 하고 있다면, 디코더에서 생성한 한글 단어가 query를 던지고, 여기에 반응(key, value)은 영어 단어로부터 온다.)
생성되고 있는 한국어 단어에 대한 어텐션 정보는 인코더에 있는 영어 단어로부터 와야 한다.
feed-forward-neural-network와 잔차연결&정규화는 encoder와 동일하게 수행된다.
마지막 단계에서 선형변환과 softmax 층은 다음 단어를 예측하기 위해 사용된다.


## 2. BERT의 이해와 활용

BERT(Bidirectional Encoder Representations form Trnasformers)는 현재 시점에서 텍스트 마이닝 딥러닝 모형 중 가장 중요한 모형이다.
텍스트 마이닝의 전분야에서 우수한 성능을 보여주고 있다.

- 언어 모델과 학습법
- BERT의 구조
- 기본 활용 방법

#### 언어 모델의 장점

비지도 학습이 가능한 것

트랜스포머로 번역 모델을 학습한다고 하면, 반드시 번역의 대상이 되는 문장과 번역한 문장이 쌍으로 존재해야한다. (답이 무조건 존재해야 한다.)
언어모델은 주어진 단어의 시퀀스 뒤에 오는 단어를 예측하는 방식으로 학습하므로, 문장에 대한 답을 별도로 필요하지 않고 문서를 적절하게 잘라서
학습이 가능하다.

→ 우리는 사전에 언어모델을 이용해 미리 학습된 모델을 사용한다.

#### 사전학습 언어모델의 이론적 이해

언어모델 : 단어의 시퀀스에 대해 확률을 할당하는 모델
주어진 입력 단어 시퀀스(입력 문장)에 대해 대상으로 하는 **출력 단어 시퀀스(출력 문장/ 하나의 단어 값 혹은 단어의 시퀀스)의 확률을 추정하는 것**으로
하나의 단어가 아닌 단어 시퀀스, 즉 문장에 대해 확률을 할당하는 언어모델이 된다.

언어모델은 자연어문장을 이용해 주어진 단어 시퀀스에 대해 다음 단어를 예측하도록 하는 데이터셋을 자동으로 생성함으로써 학습을 수행한다.


### Bert의 구조

Bert(Bidirectional Encoder Representations from Transformers)는 언어 모델 기반의 학습을 이용해 언어에 대한 이해를 높이는 데 목적이 있다.
Transformer의 앞 부분인 Encoder만 사용한 모형으로, 이 Encoder는 양방향(Bidirectional) self-attention을 구현하고 있다.

#### GPT VS BERT
GPT는 Transformer의 Decoder를 사용하여 문장 생성에 특화된 모델이다. 따라서 Encoder에서 Decoder로의 Attention은 생략되어 있고, self-attention의 순방향만 적용된다.
반면, Bert는 Transformer의 Encoder 부분만 떼어서 사용을 하였는데, 양방향 self-attention을 모두 활용할 수 있다.

#### 언어모델을 이용한 사전학습과 미세조정 학습

BERT의 학습은 사전학습과 미세조정학습의 두 단계로 나누어진다.

사전학습 : 언어에 대한 이해를 높이기 위한 비지도학습

Q. Bert는 어떻게 학습을 수행하는 것인가?

첫번째 방법 1. masking이라는 기법을 사용하여 모델을 학습시킨다. 즉, 단어를 가리고 가린 단어를 예측하게 한다.
즉 순서와 관계없이, 문장 안에서 **랜덤한 위치의 단어를 지우고** 모형이 이 단어들을 예측하도록 한다.
가려진 단어는 문장의 중간에 위치하며, 양쪽에 단어들이 있어 양방향 셀프 어텐션을 모두 이용해 예측하는 것이 가능하다.

두번째 방법 2. 두 문장을 다루어 두 문장이 같은 의미인지 혹은 앞 문장의 내용으로 뒤 문장의 질문에 대한 답이 가능한지 등을 알아본다.
Transformer는 Encoder와 Decoder로 구성되어 있어, 두 문장을 다루어야 하는 경우에 하나는 인코더로, 하나는 디코더로 보내는 것이지만,
Bert는 Encoder밖에 없어 두 문장을 구분하는 토큰을 정의하고 두 문장 사이에 넣어서 하나의 시퀀스를 만든 후에 Encoder에서 한 번에 처리한다.

미세조정학습 : 실제 수행하고자 하는 작업에 대한 지도학습


#### 실습

<Transformer로 수행할 수 있는 작업>

transformers 라이브러리를 사용해 파이프라인을 이용하면 된다.

1. 감정분석
2. 문서분류
3. 질의응답
4. 문서생성
5. 기계번역
6. 문서요약

이래의 파이프라인에서 사용한 모형 : https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english)

In [2]:
from transformers import pipeline

clf = pipeline("sentiment-analysis")
result = clf("What a beautiful day!")[0]
print("#감정 분석 결과 : %s, 감정 스코어 : %0.4f"%(result['label'], result['score']))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahyeo\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

#감정 분석 결과 : POSITIVE, 감정 스코어 : 0.9999


In [3]:
from transformers import pipeline

text_generator = pipeline('text-generation')
result = text_generator("Alice was beginning to get very tired of sitting by her sister on the bank, ")
print(result[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahyeo\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Alice was beginning to get very tired of sitting by her sister on the bank,  he thought. He went back to the hotel, but did not get into trouble before she turned back around. She was very friendly, friendly with the guests,


In [5]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# 토크나이저와 모델 로드
# BERT의 모형과 학습 ㅇ ㅝㄴ리에 최적화된 토큰화 방식을 사용한다.
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

# 입력 문장과 타겟 시퀀스
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner" #input_sentence의 요약 버전

# 토큰화
tokens = tokenizer(input_sentence, target_sequence, return_tensors="tf")

# 모델을 통한 예측 (logits는 softmax를 적용하기 이전의 신경망 결과)
logits = model(**tokens).logits

# Softmax를 사용하여 로짓을 확률로 변환 (확률값을 구하기 위해서는 softmax 적용 필수)
results = tf.nn.softmax(logits, axis=-1).numpy()[0]

# 결과 출력
for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


no: 29%
yes: 71%


In [7]:
# 상관없는 문장으로 다시 한 번 수행
target_sequence = "The boy quickly ran across the finish line, seizing yet another victory"
tokens = tokenizer(input_sentence, target_sequence, return_tensors='tf')

logits = model(**tokens).logits

results = tf.nn.softmax(logits, axis=-1).numpy()[0]

# 결과 출력
for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 95%
yes: 5%


#### 영화리뷰 데이터셋 사용

In [8]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

nltk.download("movie_reviews")

fileids = movie_reviews.fileids()

reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

label_dict = {'pos' : 0, 'neg' : 1}
y = np.array([label_dict[c] for c in categories])

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print("#Train set count : ", len(X_train))
print("#Test set count : ", len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ahyeo\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


#Train set count :  1600
#Test set count :  400


In [10]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
import numpy as np

# GPU 사용 가능 여부 확인
if tf.config.experimental.list_physical_devices('GPU'):
    device = '/GPU:0'
else:
    device = '/CPU:0'

#BERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

batch_size = 10 #모형으로 한 번에 예측할 데이터의 수 (메모리를 많이 잡아먹어서 이렇게 수행)
y_pred = [] #예측 결과 저장

num_batch = len(y_test) // batch_size

for i in range(num_batch):
    inputs = tokenizer(
        X_test[i * batch_size:(i + 1) * batch_size],
        truncation=True,
        padding=True,
        return_tensors="tf"
    )

    logits = model(**inputs).logits
    pred = tf.nn.softmax(logits, axis=-1)
    results = tf.argmax(pred, axis=1).numpy()
    y_pred.extend(results.tolist())

# 메모리 관리를 위해 TensorFlow 세션 클리어
tf.keras.backend.clear_session()

score = np.sum(y_test == np.array(y_pred)) / len(y_test)
print("NLTK 영화리뷰 감성분석 정확도:", score) #왜 정확도가... 15.75%..?? -> 원래 85.75%가 나옴..

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahyeo\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


NLTK 영화리뷰 감성분석 정확도: 0.1575


### BERT 사전학습 모형에 대한 미세조정 학습

1 . 마스킹에 의한 빈 단어의 예측
2 . 두 문장의 선후관계 예측

< 사용되는 3개의 Embedding >

1 . Token Embedding
토큰 임베딩에는 문장에 사용된 단어 외에 두 종류의 특수 토큰이 추가된다. [CLS] 토큰과 [SEP] 토큰
[CLS] 토큰 : 한 문서에 대한 문서 분류나 두 문서의 관계에 대한 분류를 하기 위한 정보를 수집해 최종적으로 출력하는 역할
[SEP] 토큰 : seperator 토큰으로 한 문장의 끝을 나타내거나 두 문장을 분리한다.

2 . Segement Embedding
BERT에서 두 문장을 하나의 시퀀스로 만들어 입력으로 사용하므로, 각 토큰이 어느 문장에 속하는지 별도로 임베딩한다.

3 . Position Embedding
시퀀스에서의 순서를 나타낸다.


BertTokenizer : 가장 기본적인 클래스
bert-base-uncased : 미세조정학습 없이 언어모델에 대한 사전학습만 수행한 모형


In [11]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"

print(sentence1, '토큰화 결과', tokenizer.tokenize(sentence1))
print(sentence2, '토큰화 결과', tokenizer.tokenize(sentence2))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahyeo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

What a beautiful day! 토큰화 결과 ['what', 'a', 'beautiful', 'day', '!']
Nvidia Titan XP has 12GB of VRAM 토큰화 결과 ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [12]:
inputs = tokenizer([sentence1, sentence2], padding = True)
print("Bert 입력 : ", inputs)

Bert 입력 :  {'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [13]:
inputs = tokenizer(sentence1, sentence2, padding = True)
print("두 문장 시퀀스에 대한 BERT 입력 : ", inputs)

두 문장 시퀀스에 대한 BERT 입력 :  {'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [2]:
## 미세조정학습
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ahyeo\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count:  1600
Test set count:  400


In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="tf")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="tf")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahyeo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buf

In [5]:
import tensorflow as tf

def create_dataset(inputs, labels):
    # 입력과 레이블을 텐서로 변환
    inputs_tensors = {key: tf.convert_to_tensor(val) for key, val in inputs.items()}
    labels_tensors = tf.convert_to_tensor(labels)

    # 텐서로부터 데이터셋 생성
    dataset = tf.data.Dataset.from_tensor_slices((inputs_tensors, labels_tensors))
    return dataset

# 훈련 및 테스트 데이터셋 생성
train_dataset = create_dataset(train_input, y_train)
test_dataset = create_dataset(test_input, y_test)

In [8]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(predictions, labels):
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\ahyeo\anaconda3\envs\TextMining\lib\site-packages\datasets\load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [7]:
#이건 PyTorch 기반임..
# from datasets import load_metric
#
# metric = load_metric("accuracy")
#
# def compute_metrics(predictions, labels):
#     predictions = np.argmax(predictions, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

# from transformers import Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir = './results', #체크포인트 출력 폴더
#     num_train_epochs = 2, #학습 수
#     per_device_train_batch_size = 8, #학습에서 사용할 배치의 크기 (BERT는 메모리를 많이 차지하기 때문에 배치의 사이즈가 매우 적다.)
#     per_device_eval_batch_size=16 #예측에서 사용할 배치의 크기
# )
#
# trainer = Trainer(
#     model = model, #학습할 모형
#     args = training_args, #위에서 정의한 학습 매개변수들
#     train_dataset = train_dataset, #학습데이터셋
#     compute_metrics=compute_metrics #검증 방식
# )
#
# #미세조정학습 실행
# trainer.train()

ImportError: 
Trainer requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFTrainer".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [9]:
#tensorflow 기반으로 수정
# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 커스텀 콜백 정의
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # 검증 데이터셋에 대한 예측
        predictions = self.model.predict(test_dataset.map(lambda x, y: x))
        # compute_metrics 함수를 사용하여 메트릭 계산
        eval_metrics = compute_metrics(predictions, np.concatenate([y for x, y in test_dataset], axis=0))
        print(f"\nEpoch {epoch+1} Eval Metrics:", eval_metrics, "\n")

# 모델 학습
model.fit(train_dataset.shuffle(1000).batch(8),
          epochs=2,
          validation_data=test_dataset.batch(16),
          callbacks=[CustomCallback()])

Epoch 1/2
  4/200 [..............................] - ETA: 1:49:53 - loss: 0.8092 - accuracy: 0.5000

KeyboardInterrupt: 

In [ ]:
results = model.evaluate(test_dataset)
print("test loss, test acc:", results)

GPU가 없어서 시간 관계상 epoch 멈춤.